In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from transformers import TFBertModel, BertConfig, AutoTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

In [ ]:
# 사용 가능한 GPU 목록 확인
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))
print("GPU Details:", gpus)

### 데이터 전처리

In [ ]:
train_df = pd.read_csv('../data/train.csv', index_col=0) # 캐글 데이터
gen_df = pd.read_csv('../data/gen_data_final998.csv', index_col=0) # 합성데이터
class_dict = {'협박 대화': 0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [ ]:
gen_df['topic'] = '일반 대화'
gen_df = gen_df.rename(columns={'topic':'class'})

In [ ]:
data_df = pd.concat([train_df, gen_df], ignore_index=True)

In [ ]:
data_df['class'] = data_df['class'].apply(lambda x: class_dict[x])

In [ ]:
# # train 데이터의 최대 길이를 구함
# data_len = [len(x.split()) for x in data_df['conversation']]
# MAX_LEN = max(data_len)
# MAX_LEN

In [ ]:
labels = list(data_df['class'])
len(data_df['conversation']), len(labels) # 대화 , labels 갯수 확인

In [ ]:
np.unique(labels) # 레이블 집합

In [ ]:
num_classes = len(np.unique(labels))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # klue 토크나이저 사용

In [ ]:
token_data = tokenizer(
    list(data_df['conversation']),
    padding='max_length', # 자동으로 최대 길이로 패딩해줌
    truncation=True, # 모델이 감당 가능한 최대 길이 초과하면 자름
    return_tensors='np'
)

In [ ]:
# 토크나이저에 padding 옵션을 넣어줘서
lengths = [len(seq) for seq in token_data['input_ids']]
print(f"토크나이저 후 데이터 내 최대 시퀀스 길이: {max(lengths)}")

In [ ]:
for key, train in token_data.items():
    print(key, train)
    break

### 모델

In [44]:
num_samples = len(data_df) # 전체 샘플 갯수
indices = np.arange(num_samples) # 인덱스 생성

train_indices, val_indices = train_test_split( # 인덱스를 8대2로 나눔
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels # stratify에는 target값으로 class 비율 일정하게 셔플
)

In [ ]:
train_inputs = {key: tf.gather(train, train_indices) for key, train in token_data.items()}
val_inputs = {key: tf.gather(val, val_indices) for key, val in token_data.items()}

# 레이블도 동일한 인덱스로 선택
train_labels = tf.gather(labels, train_indices)
val_labels = tf.gather(labels, val_indices)

train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(8) # 셔플 및 배치

# 예시: 검증 데이터셋 생성
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels))
val_dataset = val_dataset.batch(8) # 검증 데이터는 보통 셔플하지 않음

In [ ]:
for i in train_dataset:
    print(i)
    break

In [47]:
config = BertConfig(
    vocab_size=tokenizer.vocab_size,  # BERT의 기본 vocabulary 크기
    hidden_size=768,   # hidden size (BERT base model의 경우 768)
    num_attention_heads=4,  # Attention heads 수 (BERT base 모델의 경우 12)
    num_hidden_layers=8,    # hidden layers 수 (BERT base 모델의 경우 12)
    intermediate_size=3072,  # 중간 차원 크기
    num_labels = num_classes,    # 분류할 클래스 수
    max_position_embeddings=512,  # 최대 시퀀스 길이
    type_vocab_size=2,  # 세그먼트 유형 (BERT에서는 2개, 0과 1)
    attention_probs_dropout_prob=0.4,  # Attention dropout
    hidden_dropout_prob=0.4,  # Hidden layer dropout
    initializer_range=0.02,  # 가중치 초기화 범위
)

# BERT 모델 초기화 (처음부터 학습)
model = TFBertForSequenceClassification(config)

In [48]:
for layer in model.layers:
    print(f"Layer {layer.name} is trainable: {layer.trainable}")

Layer bert is trainable: True
Layer dropout_77 is trainable: True
Layer classifier is trainable: True


In [49]:
optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 10. 모델 컴파일
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [50]:
# 4. 더미 입력으로 모델 빌드
dummy_input = tokenizer(
    ["더미 텍스트"],
    padding='max_length', # 최대 길이 통일하기 위한 패딩
    truncation=True, # 모델이 감당 가능한 최대 길이 초과하면 자름
    return_tensors='tf'
)
model(dummy_input)  # 모델 호출로 빌드

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-0.09326675, -0.02177519, -0.18035454, -0.11851996,  0.42093703]],
      dtype=float32)>, hidden_states=None, attentions=None)

In [51]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  82265856  
_________________________________________________________________
dropout_77 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 82,269,701
Trainable params: 82,269,701
Non-trainable params: 0
_________________________________________________________________


In [26]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    restore_best_weights=True,
    patience=2)

# ModelCheckpoint 콜백 수정
model_checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath='vanilla_weight.h5', # 파일 확장자를 .keras (권장) 또는 .h5 로 지정
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,      # 전체 모델 저장 (기본값이므로 생략 가능)
    verbose=1
)

In [27]:
NUM_EPOCHS = 50

In [28]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS,
    verbose=1,
    callbacks=[early_stopping_cb] # 정의된 콜백 전달
    # callbacks=[early_stopping_cb, model_checkpoint_cb] # 정의된 콜백 전달
)

Epoch 1/50
  2/495 [..............................] - ETA: 3:53:34 - loss: 1.6635 - accuracy: 0.2500

KeyboardInterrupt: 

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors='tf', padding='max_length', truncation=True)
    logits = model(inputs).logits
    return int(tf.argmax(logits, axis=1).numpy()[0])

In [ ]:
test_df = pd.read_csv('../data/test.csv', index_col=0)
submission = pd.read_csv('../data/submission.csv', index_col=0)

In [ ]:
submission['target'] = test_df['text'].apply(predict)

In [ ]:
submission.to_csv('bert_vanilla_sub.csv')